In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
from sympy.matrices import Matrix, eye
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.message import Message, SOMStoichiometry, SOMReaction

from scipy.linalg import lu, inv

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# make sure to run #BIOMD 18
# #112 is very tricky to detect errors (no errors?!)
# Biomodels 383 can be used to test reduction 
simple = load_file_from_curated_data(120)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

v2: lck_active -> lck_inactive
v3: lck_inactive -> lck_active
v4: lck_inactive -> lck_active
v7: phosphatase_inactive -> phosphatase_active
v8: phosphatase_inactive -> phosphatase_active
v9: phosphatase_active -> phosphatase_inactive


In [3]:
m = Message(simple)

In [4]:
sm = m.stoichiometry_matrix.T

In [5]:
m.analyze()

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
RREF error:  []
Type III error:  []
Type I-SOM error:  set()


False

In [6]:
m.som_stoichiometry_matrix

In [8]:
m.reactions_lu

[]

In [8]:
m.som_stoichiometry_matrix

,reaction_6,reaction_7,reaction_2
{Pi},0.0,1.0,0.0
{R_smad_P_nuc},1.0,-1.0,0.0
"{R_smad_P_cyt, R_smad_cyt, R_smad_nuc}",0.0,1.0,-1.0
"{smad4_cyt, smad4_nuc}",1.0,0.0,-1.0
"{R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}",-1.0,0.0,1.0


In [14]:
m.decomposeMatrix(sm).T

,{Pi},{R_smad_P_nuc},"{R_smad_P_cyt, R_smad_cyt, R_smad_nuc}","{smad4_cyt, smad4_nuc}","{R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}"
reaction_7,1.0,-1.0,1.0,0.0,0.0
reaction_6,0.0,1.0,0.0,1.0,-1.0
reaction_2,0.0,0.0,-1.0,-1.0,1.0


In [11]:
m.edges

OutEdgeView([({Pi}, {R_smad_P_nuc}), ({R_smad_P_nuc}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}), ({R_smad_P_cyt, R_smad_cyt, R_smad_nuc}, {R_smad_P_nuc}), ({R_smad_P_cyt, R_smad_cyt, R_smad_nuc}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}), ({smad4_cyt, smad4_nuc}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc})])

In [12]:
nx.simple_cycles(m.edges)

AttributeError: 'OutEdgeView' object has no attribute 'is_directed'

In [25]:
m.processUnequalSOMReaction(m.reduced_som_reactions[0])
m.edges

OutEdgeView([({CO2}, {PGA, RuBP}), ({NADP, NADPH}, {PGA, RuBP})])

In [22]:
reaction = m.reduced_som_reactions[0]
category = reaction.category
if (category!=cn.REACTION_n_1) or (category!=cn.REACTION_1_n):
  print("noo")
# else:
#   if category == cn.REACTION_n_1:
#     destination = [reaction.products[0].molecule]
#     source = [reactant.molecule for reactant in reaction.reactants]
#   elif category == cn.REACTION_1_n:
#     destination = [reaction.reactants[0].molecule]
#     source = [product.molecule for product in reaction.products]
#   arcs = itertools.product(source, destination)
#   for arc in arcs:
#     if arc[0] == arc[1]:
#       self.type_one_som_errors = self.type_one_som_errors.add(reaction.label)
#     else:
#       self.addArc(arc[0], arc[1], reactin.label)

noo


In [13]:
m.lower_inverse.dot(m.permuted_matrix.T)

,{CO2},"{PGA, RuBP}","{NADP, NADPH}"
0,-1.0,1.0,-2.0
1,-2.0,1.5,-4.0


In [14]:
p, l, u = lu(m.stoichiometry_matrix.T)
p

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [13]:
l

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  1.,  0.],
       [ 0., -1.,  0., -1., -1., -1.,  1.]])

In [14]:
u

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [15]:
l.dot(u)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.]])

In [16]:
# this is the matrix for operation (without transpose)
reduce_operation = inv(l).dot(inv(p))
reduce_operation

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [15]:
print(m.analyze())
m.rref_errors

False


[]

In [6]:
if [] or [] or []:
  print("none")

In [10]:
rref_df = m.decomposeMatrix(m.stoichiometry_matrix)
reactions = m.convertMatrixToReactions(simple, rref_df)

In [11]:
reactions

[ReactionSummary(label='reaction6', reactants=[XI *  1.00], products=[X *  1.00], category='reaction_1_1'),
 ReactionSummary(label='reaction5', reactants=[M *  1.00], products=[MI *  1.00], category='reaction_1_1'),
 ReactionSummary(label='reaction4', reactants=[], products=[], category='reaction_redundant'),
 ReactionSummary(label='reaction7', reactants=[], products=[], category='reaction_redundant')]

In [12]:
k = m.processUniUniReaction(reactions[0])

In [13]:
type(reactions[0].reactants[0].molecule)

SBMLLint.common.molecule.Molecule

In [14]:
str(reactions[0].label)

'reaction6'

In [15]:
m2 = MESGraph(simple)
m2.analyze()

Model analyzed...
No error detected.


''

In [16]:
s = StoichiometryMatrix(simple)
s.isConsistent()

True

In [17]:
list(m2.nodes)[1].reactions

{reaction4: MI -> M; cell * MI * V1 * pow(K1 + MI, -1)}